In [5]:
import cv2
import numpy as np

image_path = 'Images/image1x1.png'

def find_rectangles(image_path):
    # Read the image
    image = cv2.imread(image_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Optional: Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Detect edges using the Canny algorithm
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    
    # Find contours from the edged image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    # List to hold coordinates of rectangles
    rectangles = []
    
    for cnt in contours:
        # Approximate the contour to a polygon
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.05 * peri, True)
        
        # The contour is a rectangle if it has 4 points
        if len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            rectangles.append((x, y, w, h))
            
            # Draw the rectangle on the image
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Save or display the image with rectangles drawn
    cv2.imwrite('rectangles_found.png', image)
    
    return rectangles

# Replace 'image.png' with the path to your image file
rectangles = find_rectangles('Images/image1x1.png')
print(f"Found rectangles: {rectangles}")

#using cv to generate images of the rectangles
for i, rect in enumerate(rectangles):
    x, y, w, h = rect
    img = cv2.imread('Images/image1x1.png')
    cv2.imwrite(f'rectangle_{i}.png', img[y:y+h, x:x+w])
    print(f"Saved rectangle_{i}.png")
    
#generating the image with the circled rectangles
img = cv2.imread('Images/image1x1.png')
for rect in rectangles:
    x, y, w, h = rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imwrite('rectangles_circled.png', img)
print(f"Saved rectangles_circled.png")


Found rectangles: [(2022, 2552, 9, 6), (2023, 2549, 8, 6), (2024, 2534, 9, 8), (805, 2476, 3, 3), (826, 2460, 5, 6), (2019, 2428, 3, 3), (2036, 2389, 14, 6), (2042, 2378, 13, 9), (2209, 2349, 12, 7), (2201, 2336, 9, 6), (2227, 2322, 10, 7), (915, 2291, 8, 5), (898, 2285, 14, 10), (897, 2285, 16, 9), (892, 2284, 4, 6), (899, 2274, 3, 4), (893, 2270, 2, 6), (913, 2268, 8, 8), (898, 2265, 7, 7), (156, 2256, 5, 5), (406, 2255, 6, 5), (413, 2247, 9, 8), (417, 2242, 6, 5), (398, 2236, 11, 9), (396, 2236, 13, 8), (420, 2223, 8, 5), (395, 2216, 6, 5), (395, 2216, 6, 5), (154, 2204, 5, 5), (2229, 2190, 3, 2), (2210, 2172, 17, 12), (2210, 2172, 19, 12), (2210, 2166, 8, 6), (374, 2145, 5, 5), (397, 2140, 11, 8), (394, 2132, 7, 7), (382, 2122, 8, 8), (377, 2119, 12, 12), (939, 1895, 12, 8), (938, 1895, 13, 7), (952, 1886, 9, 5), (942, 1878, 5, 7), (942, 1878, 5, 7), (961, 1869, 6, 6), (952, 1866, 7, 5), (1872, 1802, 5, 5), (1872, 1802, 5, 5), (2035, 1790, 9, 8), (2057, 1775, 8, 8), (1422, 1774, 7,

In [2]:
import cv2
import numpy as np

# Load the image
file_path = 'Images/image1x1.png'
image = cv2.imread(file_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a blur to reduce noise
blurred = cv2.GaussianBlur(gray, (9, 9), 0)

# Threshold the image
_, thresh = cv2.threshold(blurred, 10, 255, cv2.THRESH_BINARY)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Define a function to check if a contour is a rectangle
def is_rectangle(cnt, eps_ratio=0.05):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, eps_ratio * peri, True)
    if len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        aspect_ratio = float(w)/h if h > 0 else 0
        if 0.8 < aspect_ratio < 1.2:  # More flexible on aspect ratio
            return True
    return False

# Filter for rectangles
rectangles = [cnt for cnt in contours if is_rectangle(cnt)]

# Draw rectangles on the image
for cnt in rectangles:
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

#generating a new image with rectangles
cv2.imwrite('rectangles_found2.png', image)


True

In [3]:
import cv2
import numpy as np

# Load the image again to adjust the detection
file_path = 'Images/image1x1.png'
image = cv2.imread(file_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a blur to reduce noise
blurred = cv2.GaussianBlur(gray, (9, 9), 0)

# Apply adaptive thresholding to accommodate varying lighting conditions
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Dilate to combine adjacent contours
kernel = np.ones((3, 3), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Function to filter for rectangles based on contour approximation
def is_rectangle(cnt, eps_ratio=0.05):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, eps_ratio * peri, True)
    if len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        aspect_ratio = float(w)/h if h > 0 else 0
        if 0.8 < aspect_ratio < 1.2:  # More flexible on aspect ratio
            return True
    return False

# Filter for rectangles
rectangles = [cnt for cnt in contours if is_rectangle(cnt)]

# Draw rectangles on the image
for cnt in rectangles:
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# generating a new image with the rectangles 
cv2.imwrite('rectangles_found3.png', image)


True

In [4]:

# Load the image again for the new detection process

file_path = 'Images/image1x2.png'
image = cv2.imread(file_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a blur to reduce noise
blurred = cv2.GaussianBlur(gray, (9, 9), 0)

# Apply adaptive thresholding to accommodate varying lighting conditions
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Define a function to check if a contour is a rectangle using area and color density
def is_good_rectangle(cnt, image, min_area=100, max_area=5000, color_threshold=50):
    area = cv2.contourArea(cnt)
    if min_area < area < max_area:
        # Calculate the intensity of the color inside the contour
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [cnt], -1, 255, -1)
        mean_val = cv2.mean(image, mask=mask)[0:3]  # Only take the BGR channels
        if np.mean(mean_val) > color_threshold:
            return True
    return False

# Filter for good rectangles
good_rectangles = [cnt for cnt in contours if is_good_rectangle(cnt, image)]

# Draw rectangles on the image
for cnt in good_rectangles:
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

#generating the result 
cv2.imwrite('rectangles_found4.png', image)


True